# Análisis de adjudicación de proyectos de I+D por Universidades en Chile

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />


In [ ]:
import numpy as np
import pandas as pd

from fuzzywuzzy import fuzz
import re, unicodedata, datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('max_colwidth', 200)

In [ ]:
mineduc_file='data/Mineduc Listado IES Vigentes 05-2016.csv'
mineduc=pd.read_csv(mineduc_file, header=0, sep=',')
mineduc['fecha_reconocimiento']=pd.to_datetime(mineduc['fecha_reconocimiento'])
mineduc['año_reconocimiento']=mineduc['fecha_reconocimiento'].dt.year

In [ ]:
mineduc.head()

In [ ]:
for i in range(len(mineduc.index)):
    old=mineduc.loc[i,'nombre']
    new=re.sub(" ?\(\+\+\) ?| ?\(\*.*\) ?","", old)
    print("OLD: ", old)
    print("NEW: ", new)
    mineduc.loc[i,'nombre']=new

Usaremos solamente las Universidades tipo A, B y C

In [ ]:
mineduc_univ=mineduc.query("tipo=='A' | tipo=='B' | tipo=='C'")
mineduc_univ=mineduc_univ.loc[mineduc_univ["nombre"].str.lower().sort_values(ascending=True).index]
mineduc_univ['nombre']
#['nombre'].sort_values(axis=0, ascending=False)

## Cargamos los archivos Fondecyt regular

Estos archivos vienen en format csv. Los nombres que aparecen en la columna nombre vienen en mayúsculas y no son completamente oficiales.

El objetivo es hacer un match entre la columna nombre de estos archivos y la columna nombre de Mineduc para así quedarnos con el nombre oficial reconocido por Mineduc

In [ ]:
fondecyt_2010=pd.read_csv('data/fondecyt/fondecyt regular 2010.csv', header=0, sep=',')
fondecyt_2011=pd.read_csv('data/fondecyt/fondecyt regular 2011.csv', header=0, sep=',')
fondecyt_2012=pd.read_csv('data/fondecyt/fondecyt regular 2012.csv', header=0, sep=',')
fondecyt_2013=pd.read_csv('data/fondecyt/fondecyt regular 2013.csv', header=0, sep=',')
fondecyt_2014=pd.read_csv('data/fondecyt/fondecyt regular 2014.csv', header=0, sep=',')
fondecyt_2015=pd.read_csv('data/fondecyt/fondecyt regular 2015.csv', header=0, sep=',')
fondecyt_2016=pd.read_csv('data/fondecyt/fondecyt regular 2016.csv', header=0, sep=',')

In [ ]:
fondecyt_2010.head()

Reemplazamos algunas abreviaciones por el nombre completo

In [ ]:
fondecyt_regular=pd.concat([fondecyt_2010,fondecyt_2011,fondecyt_2012,fondecyt_2013,fondecyt_2014,fondecyt_2015,fondecyt_2016]).reset_index(drop=True)
for i in range(len(fondecyt_regular.index)):
    name=(fondecyt_regular.loc[i,'nombre']).upper()
    name=name.replace("UNIV.","UNIVERSIDAD").replace("PONT.","PONTIFICIA").replace("CS.","CIENCIAS").replace("TEC.","TECNOLOGIA")
    name=name.replace("INTERNACIONAL SEK","SEK").replace("UNIVERSIDAD CIENCIAS DE LA INFORMATICA","UNIVERSIDAD UCINF")
    fondecyt_regular.loc[i,'nombre']=name.capitalize()

fondecyt_regular.head()

Hacemos el match usando el paquete fuzzyword

In [ ]:
for i in range(len(fondecyt_regular.index)):
    if fondecyt_regular.loc[i,'nombre'] != 'Otras universidades':
        d = mineduc.apply(lambda x: fuzz.ratio(x['nombre'].upper(), fondecyt_regular.loc[i, 'nombre'].upper()), axis=1)
        if d.max()<90:
            print("WARNING - Fuzz ratio lower than 90")
            print('Fondecyt name: ', fondecyt_regular.loc[i,'nombre'])
            print('Mineduc name: ', mineduc.loc[d.idxmax(), 'nombre'])
        fondecyt_regular.loc[i,'nombre']=mineduc.loc[d.idxmax(), 'nombre']

fondecyt_regular.head()

## Realizamos algunas operaciones de agregación

Agrupemos por nombre de Universidad

In [ ]:
fondecyt_univ = fondecyt_regular.groupby(['nombre']).sum().sort_values('n_aprobados', ascending=False)
fondecyt_univ.head()

In [ ]:
fondecyt_univ.to_csv('results/Fondecyt_agrupado_por_universidades', index=False)

Agrupemos por nombre de Universidad y año de adjudicación

In [ ]:
fondecyt_univ_year=fondecyt_regular.groupby(['nombre','año']).sum()  #.sort_values('n_aprobados',ascending=False)[['n_concursados','n_aprobados']]
fondecyt_univ_year['tasa_aprobacion']=np.round(fondecyt_univ_year['n_aprobados']/fondecyt_university['n_concursados']*100.,decimals=1)
fondecyt_univ_year.head()

Eliminamos las Universidades con nombre "Otras Universidades"

In [ ]:
fondecyt_regular = fondecyt_regular[fondecyt_regular['nombre']!= "Otras universidades"]
fondecyt_regular.head()

In [ ]:
fondecyt_univ_year=fondecyt_regular.groupby(['nombre','año']).sum()  #.sort_values('n_aprobados',ascending=False)[['n_concursados','n_aprobados']]
fondecyt_univ_year['tasa_aprobacion']=np.round(fondecyt_univ_year['n_aprobados']/fondecyt_university['n_concursados']*100.,decimals=1)
fondecyt_univ_year.head(10)

Algunas veces es conveniente tener los años como campos de la tabla

In [ ]:
table = pd.pivot_table(fondecyt_regular,values=['n_concursados','n_aprobados'],index=['nombre'],columns=['año'],aggfunc=np.sum,margins=True)
#table = table.round({'tasa_aprobacion':1})
table.sort_values(by=('n_aprobados','All'), ascending=False,inplace=True)
table.reset_index( inplace=True)
table
